<a href="https://colab.research.google.com/github/jfink09/Algorithmic-Trading/blob/main/HVC_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import yfinance as yf
except:
  print(f"[INFO] yfinance not installed...Installing it now")
  !pip install yfinance
  import yfinance as yf

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [47]:
# Download SPY daily data
ELF = yf.download("ELF",start=None,end=None,progress=False)
ELF

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-09-22,24.000000,27.400000,23.730000,26.500000,26.500000,13591500
2016-09-23,25.670000,26.049999,24.129999,25.330000,25.330000,2152500
2016-09-26,25.379999,25.760000,24.719999,25.260000,25.260000,531800
2016-09-27,25.450001,25.500000,24.570000,24.600000,24.600000,976700
2016-09-28,24.950001,26.799999,24.750000,26.540001,26.540001,1071400
...,...,...,...,...,...,...
2022-12-28,55.380001,55.655998,54.750000,55.590000,55.590000,414000
2022-12-29,56.000000,56.518002,55.009998,55.680000,55.680000,550200
2022-12-30,55.320000,55.459999,54.570000,55.299999,55.299999,401700


In [ ]:
# Import ta-lib and a backtesting library
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib
!pip install backtesting

In [5]:
from backtesting import Backtest, Strategy
#from backtesting.test import SPY
from backtesting.lib import crossover
import talib as tb

/usr/local/lib/python3.8/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [49]:
ELF

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-09-22,24.000000,27.400000,23.730000,26.500000,26.500000,13591500
2016-09-23,25.670000,26.049999,24.129999,25.330000,25.330000,2152500
2016-09-26,25.379999,25.760000,24.719999,25.260000,25.260000,531800
2016-09-27,25.450001,25.500000,24.570000,24.600000,24.600000,976700
2016-09-28,24.950001,26.799999,24.750000,26.540001,26.540001,1071400
...,...,...,...,...,...,...
2022-12-28,55.380001,55.655998,54.750000,55.590000,55.590000,414000
2022-12-29,56.000000,56.518002,55.009998,55.680000,55.680000,550200
2022-12-30,55.320000,55.459999,54.570000,55.299999,55.299999,401700


In [50]:
# Declare a list that is to be converted into a column
pct_chg = ELF["Close"].pct_change() * 100
  
# Using 'Address' as the column name
# and equating it to the list
ELF['pct_chg'] = pct_chg
ELF

,Open,High,Low,Close,Adj Close,Volume,pct_chg
Date,,,,,,,
2016-09-22,24.000000,27.400000,23.730000,26.500000,26.500000,13591500,NaN
2016-09-23,25.670000,26.049999,24.129999,25.330000,25.330000,2152500,-4.415095
2016-09-26,25.379999,25.760000,24.719999,25.260000,25.260000,531800,-0.276351
2016-09-27,25.450001,25.500000,24.570000,24.600000,24.600000,976700,-2.612826
2016-09-28,24.950001,26.799999,24.750000,26.540001,26.540001,1071400,7.886181
...,...,...,...,...,...,...,...
2022-12-28,55.380001,55.655998,54.750000,55.590000,55.590000,414000,0.198271
2022-12-29,56.000000,56.518002,55.009998,55.680000,55.680000,550200,0.161900
2022-12-30,55.320000,55.459999,54.570000,55.299999,55.299999,401700,-0.682473


In [51]:
# Create condition to confirm if the daily percent change is at least 2%
def yesno(row):
    if row['pct_chg'] >= 2:
        val = "Yes"
    else:
        val = "No"
    return val

In [52]:
ELF['Above 2%'] = ELF.apply(yesno, axis=1)
ELF

,Open,High,Low,Close,Adj Close,Volume,pct_chg,Above 2%
Date,,,,,,,,
2016-09-22,24.000000,27.400000,23.730000,26.500000,26.500000,13591500,NaN,No
2016-09-23,25.670000,26.049999,24.129999,25.330000,25.330000,2152500,-4.415095,No
2016-09-26,25.379999,25.760000,24.719999,25.260000,25.260000,531800,-0.276351,No
2016-09-27,25.450001,25.500000,24.570000,24.600000,24.600000,976700,-2.612826,No
2016-09-28,24.950001,26.799999,24.750000,26.540001,26.540001,1071400,7.886181,Yes
...,...,...,...,...,...,...,...,...
2022-12-28,55.380001,55.655998,54.750000,55.590000,55.590000,414000,0.198271,No
2022-12-29,56.000000,56.518002,55.009998,55.680000,55.680000,550200,0.161900,No
2022-12-30,55.320000,55.459999,54.570000,55.299999,55.299999,401700,-0.682473,No


In [53]:
# Declare a list that is to be converted into a column
vol_pct_chg = ELF["Volume"].pct_change() * 100
  
# Using 'Address' as the column name
# and equating it to the list
ELF['vol_pct_chg'] = vol_pct_chg
ELF

,Open,High,Low,Close,Adj Close,Volume,pct_chg,Above 2%,vol_pct_chg
Date,,,,,,,,,
2016-09-22,24.000000,27.400000,23.730000,26.500000,26.500000,13591500,NaN,No,NaN
2016-09-23,25.670000,26.049999,24.129999,25.330000,25.330000,2152500,-4.415095,No,-84.162896
2016-09-26,25.379999,25.760000,24.719999,25.260000,25.260000,531800,-0.276351,No,-75.293844
2016-09-27,25.450001,25.500000,24.570000,24.600000,24.600000,976700,-2.612826,No,83.659270
2016-09-28,24.950001,26.799999,24.750000,26.540001,26.540001,1071400,7.886181,Yes,9.695915
...,...,...,...,...,...,...,...,...,...
2022-12-28,55.380001,55.655998,54.750000,55.590000,55.590000,414000,0.198271,No,5.289929
2022-12-29,56.000000,56.518002,55.009998,55.680000,55.680000,550200,0.161900,No,32.898551
2022-12-30,55.320000,55.459999,54.570000,55.299999,55.299999,401700,-0.682473,No,-26.990185


In [54]:
# Create condition to confirm if the daily volume percent change is at least 50%
def vol_yesno(row):
    if row['vol_pct_chg'] >= 50:
        val = "Yes"
    else:
        val = "No"
    return val

In [55]:
ELF['Vol Above 50%'] = ELF.apply(vol_yesno, axis=1)
ELF

,Open,High,Low,Close,Adj Close,Volume,pct_chg,Above 2%,vol_pct_chg,Vol Above 50%
Date,,,,,,,,,,
2016-09-22,24.000000,27.400000,23.730000,26.500000,26.500000,13591500,NaN,No,NaN,No
2016-09-23,25.670000,26.049999,24.129999,25.330000,25.330000,2152500,-4.415095,No,-84.162896,No
2016-09-26,25.379999,25.760000,24.719999,25.260000,25.260000,531800,-0.276351,No,-75.293844,No
2016-09-27,25.450001,25.500000,24.570000,24.600000,24.600000,976700,-2.612826,No,83.659270,Yes
2016-09-28,24.950001,26.799999,24.750000,26.540001,26.540001,1071400,7.886181,Yes,9.695915,No
...,...,...,...,...,...,...,...,...,...,...
2022-12-28,55.380001,55.655998,54.750000,55.590000,55.590000,414000,0.198271,No,5.289929,No
2022-12-29,56.000000,56.518002,55.009998,55.680000,55.680000,550200,0.161900,No,32.898551,No
2022-12-30,55.320000,55.459999,54.570000,55.299999,55.299999,401700,-0.682473,No,-26.990185,No


In [56]:
# Create condition to confirm if the daily price percent change is at least 2% and the daily volume percent change is at least 50%
def price_vol_yesno(row):
    if row['Above 2%'] == ['vol_pct_chg']:
        val = "Yes"
    else:
        val = "No"
    return val

In [57]:
ELF['Price/Vol Yes'] = ELF.apply(price_vol_yesno, axis=1)
ELF

,Open,High,Low,Close,Adj Close,Volume,pct_chg,Above 2%,vol_pct_chg,Vol Above 50%,Price/Vol Yes
Date,,,,,,,,,,,
2016-09-22,24.000000,27.400000,23.730000,26.500000,26.500000,13591500,NaN,No,NaN,No,No
2016-09-23,25.670000,26.049999,24.129999,25.330000,25.330000,2152500,-4.415095,No,-84.162896,No,No
2016-09-26,25.379999,25.760000,24.719999,25.260000,25.260000,531800,-0.276351,No,-75.293844,No,No
2016-09-27,25.450001,25.500000,24.570000,24.600000,24.600000,976700,-2.612826,No,83.659270,Yes,No
2016-09-28,24.950001,26.799999,24.750000,26.540001,26.540001,1071400,7.886181,Yes,9.695915,No,No
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28,55.380001,55.655998,54.750000,55.590000,55.590000,414000,0.198271,No,5.289929,No,No
2022-12-29,56.000000,56.518002,55.009998,55.680000,55.680000,550200,0.161900,No,32.898551,No,No
2022-12-30,55.320000,55.459999,54.570000,55.299999,55.299999,401700,-0.682473,No,-26.990185,No,No
